# Trappist 1-e models from Bower et al. (2024)

In [1]:
from atmodeller import Species, InteriorAtmosphere, Planet, earth_oceans_to_hydrogen_mass
from atmodeller import debug_logger, bulk_silicate_earth_abundances, SolverParameters
from atmodeller.solubility import get_solubility_models
from atmodeller.thermodata.redox_buffers import IronWustiteBuffer
import optimistix as optx
import logging
import numpy as np
import copy

#np.random.seed(42)
np.random.seed(0)

logger = debug_logger()
logger.setLevel(logging.INFO)
# For more output use DEBUG
#logger.setLevel(logging.DEBUG)

Atmodeller initialized with double precision (float64)


## High temperature atmospheric diversity

Parameters for the simulations

In [2]:
number_of_realisations = 10000
surface_temperature = 1800.0 # Must be float

Species to consider, where solubility is not included

In [3]:
H2O_g = Species.create_gas("H2O_g")
H2_g = Species.create_gas("H2_g")
O2_g = Species.create_gas("O2_g")
CO_g = Species.create_gas("CO_g")
CO2_g = Species.create_gas("CO2_g")
CH4_g = Species.create_gas("CH4_g")
N2_g = Species.create_gas("N2_g")
NH3_g = Species.create_gas("NH3_g")
S2_g = Species.create_gas("S2_g")
H2S_g = Species.create_gas("H2S_g")
SO2_g = Species.create_gas("SO2_g")
SO_g = Species.create_gas("SO_g")
Cl2_g = Species.create_gas("Cl2_g")

species = (H2_g, H2O_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g, S2_g, H2S_g, SO2_g, SO_g, Cl2_g)

TRAPPIST-1e planet properties

Mass and radius measurements from Agol et al. 2021; Mantle mass determined assuming same proportion as Earth

In [4]:
mantle_mass = 2.912E24
planet_mass = mantle_mass / (1-0.295334691460966)
trappist1e = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, surface_radius=5.861E6)

Earth planet properties, which are required to scale the bulk volatile inventories for Trappist-1e. Default parameters are Earth so we only need to specify the temperature.

In [5]:
earth = Planet(surface_temperature=surface_temperature)

In [6]:
earth_bse = bulk_silicate_earth_abundances()

Compute the reservoir sizes for TRAPPIST 1-e, assuming the same ppmw as Earth:

In [7]:
trappist1e_bse = copy.deepcopy(earth_bse)
mass_scale_factor = trappist1e.mantle_mass / earth.mantle_mass

for element, values in trappist1e_bse.items():
    trappist1e_bse[element] = {key: value*mass_scale_factor for key, value in values.items()}

trappist1e_bse

{'H': {'min': 1.2815326128149148e+20,
  'max': 1.3105954474467865e+21,
  'mean': 7.19374354364139e+20},
 'C': {'min': 1.2227149712980315e+20,
  'max': 2.1257387616454742e+21,
  'mean': 1.1240051293876388e+21},
 'S': {'min': 5.8236384824245807e+20,
  'max': 7.279548103030726e+20,
  'mean': 6.551593292727653e+20},
 'N': {'min': 2.417059080217331e+18,
  'max': 7.279548103030726e+18,
  'mean': 4.848303591624028e+18},
 'Cl': {'min': 5.240997845280866e+19,
  'max': 9.902122942430578e+19,
  'mean': 7.571560393855723e+19}}

In [8]:
interior_atmosphere_nosol = InteriorAtmosphere(species)

# Log uniform sampling
log10_number_oceans = np.random.uniform(-1, 1, number_of_realisations)
log10_ch_ratios = np.random.uniform(-1, 1, number_of_realisations)
fO2_log10_shifts = np.random.uniform(-5, 5, number_of_realisations)

h_kg = earth_oceans_to_hydrogen_mass(10**log10_number_oceans)
mass_constraints = {
    "H": h_kg,
    "C": h_kg * 10**log10_ch_ratios,
    "N": trappist1e_bse["N"]["mean"],
    "S": trappist1e_bse["S"]["mean"],
    "Cl": trappist1e_bse["Cl"]["mean"]
}
fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts, 1)}

# Initial solution guess number density (molecules/m^3)
# In practice, I run a smaller model to get a sense of the average number densities, and then tweak
# these estimates accordingly.
initial_log_number_density = 55 * np.ones(len(species), dtype=np.float_)
# fO2 will benefit from a smaller initial estimate
initial_log_number_density[2] = 35.0
# Initial solution guess species stability
initial_log_stability = -135 * np.ones_like(initial_log_number_density)

# Precompile
interior_atmosphere_nosol.initialise_solve(
    planet=trappist1e,
    initial_log_number_density=initial_log_number_density,
    initial_log_stability=initial_log_stability,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
)

output_nosol = interior_atmosphere_nosol.solve()

# Quick look at the solution
#solution = output.quick_look()

# Get complete solution as a dictionary
#solution_asdict = output.asdict()
#logger.info(solution_asdict)

# Write the complete solution to Excel
output_nosol.to_excel("t1e_highT_nosol")

[16:00:07 - atmodeller.classes             - INFO     ] - species = ['H2_g', 'H2O_g', 'O2_g', 'CO_g', 'CO2_g', 'CH4_g', 'N2_g', 'H3N_g', 'S2_g', 'H2S_g', 'O2S_g', 'OS_g', 'Cl2_g']
[16:00:07 - atmodeller.classes             - INFO     ] - reactions = {0: '2.0 CO2_g = 1.0 O2_g + 2.0 CO_g',
 1: '1.5 H2_g + 0.5 N2_g = 1.0 H3N_g',
 2: '2.0 H2_g + 2.0 CO_g = 1.0 CO2_g + 1.0 CH4_g',
 3: '1.0 H2_g + 0.5 S2_g = 1.0 H2S_g',
 4: '2.0 CO2_g + 0.5 S2_g = 2.0 CO_g + 1.0 O2S_g',
 5: '1.0 CO2_g + 0.5 S2_g = 1.0 CO_g + 1.0 OS_g',
 6: '1.0 H2_g + 1.0 CO2_g = 1.0 H2O_g + 1.0 CO_g'}
[16:00:10 - atmodeller.classes             - INFO     ] - Compile time: 2.967790 seconds
[16:00:10 - atmodeller.classes             - INFO     ] - Execution time: 0.483237 seconds
[16:00:10 - atmodeller.output              - INFO     ] - Creating Output
[16:00:12 - atmodeller.output              - INFO     ] - Found O2_g so back-computing log10 shift for fO2
[16:00:51 - atmodeller.output              - INFO     ] - Output writ

Cases with solubility

In [9]:
solubility_models = get_solubility_models()

In [10]:
H2O_g = Species.create_gas("H2O_g", solubility=solubility_models["H2O_basalt_dixon95"])
H2_g = Species.create_gas("H2_g", solubility=solubility_models["H2_basalt_hirschmann12"])
O2_g = Species.create_gas("O2_g")
CO_g = Species.create_gas("CO_g", solubility=solubility_models["CO_basalt_yoshioka19"])
CO2_g = Species.create_gas("CO2_g", solubility=solubility_models["CO2_basalt_dixon95"])
CH4_g = Species.create_gas("CH4_g", solubility=solubility_models["CH4_basalt_ardia13"])
N2_g = Species.create_gas("N2_g", solubility=solubility_models["N2_basalt_libourel03"])
NH3_g = Species.create_gas("NH3_g")
S2_g = Species.create_gas("S2_g", solubility=solubility_models["S2_basalt_boulliung23"])
H2S_g = Species.create_gas("H2S_g")
SO2_g = Species.create_gas("SO2_g")
SO_g = Species.create_gas("SO_g")
Cl2_g = Species.create_gas("Cl2_g", solubility=solubility_models["Cl2_basalt_thomas21"])

species = (H2_g, H2O_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g, S2_g, H2S_g, SO2_g, SO_g, Cl2_g)

In [11]:
interior_atmosphere_withsol = InteriorAtmosphere(species)

# Log uniform sampling
log10_number_oceans = np.random.uniform(-1, 1, number_of_realisations)
log10_ch_ratios = np.random.uniform(-1, 1, number_of_realisations)
fO2_log10_shifts = np.random.uniform(-5, 5, number_of_realisations)

h_kg = earth_oceans_to_hydrogen_mass(10**log10_number_oceans)
mass_constraints = {
    "H": h_kg,
    "C": h_kg * 10**log10_ch_ratios,
    "N": trappist1e_bse["N"]["mean"],
    "S": trappist1e_bse["S"]["mean"],
    "Cl": trappist1e_bse["Cl"]["mean"]
}
fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts, 1)}

# Initial solution guess number density (molecules/m^3)
# In practice, I run a smaller model to get a sense of the average number densities, and then tweak
# these estimates accordingly.
initial_log_number_density = 55 * np.ones(len(species), dtype=np.float_)
# fO2 will benefit from a smaller initial estimate
initial_log_number_density[2] = 35.0
# Initial solution guess species stability
initial_log_stability = -135 * np.ones_like(initial_log_number_density)

# Precompile
interior_atmosphere_withsol.initialise_solve(
    planet=trappist1e,
    initial_log_number_density=initial_log_number_density,
    initial_log_stability=initial_log_stability,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
)

output_withsol = interior_atmosphere_withsol.solve()

# Quick look at the solution
#solution = output.quick_look()

# Get complete solution as a dictionary
#solution_asdict = output.asdict()
#logger.info(solution_asdict)

# Write the complete solution to Excel
output_withsol.to_excel("t1e_highT_withsol")

[16:00:51 - atmodeller.classes             - INFO     ] - species = ['H2_g', 'H2O_g', 'O2_g', 'CO_g', 'CO2_g', 'CH4_g', 'N2_g', 'H3N_g', 'S2_g', 'H2S_g', 'O2S_g', 'OS_g', 'Cl2_g']
[16:00:51 - atmodeller.classes             - INFO     ] - reactions = {0: '2.0 CO2_g = 1.0 O2_g + 2.0 CO_g',
 1: '1.5 H2_g + 0.5 N2_g = 1.0 H3N_g',
 2: '2.0 H2_g + 2.0 CO_g = 1.0 CO2_g + 1.0 CH4_g',
 3: '1.0 H2_g + 0.5 S2_g = 1.0 H2S_g',
 4: '2.0 CO2_g + 0.5 S2_g = 2.0 CO_g + 1.0 O2S_g',
 5: '1.0 CO2_g + 0.5 S2_g = 1.0 CO_g + 1.0 OS_g',
 6: '1.0 H2_g + 1.0 CO2_g = 1.0 H2O_g + 1.0 CO_g'}
[16:00:54 - atmodeller.classes             - INFO     ] - Compile time: 2.993971 seconds
[16:00:54 - atmodeller.classes             - INFO     ] - Execution time: 0.387587 seconds
[16:00:54 - atmodeller.output              - INFO     ] - Creating Output
[16:00:55 - atmodeller.output              - INFO     ] - Found O2_g so back-computing log10 shift for fO2
[16:01:37 - atmodeller.output              - INFO     ] - Output writ

# Cool high temperature atmospheres

Parameters for the simulations

In [31]:
cool_surface_temperature = 280.0 # Must be a float

In [32]:
H2O_g = Species.create_gas("H2O_g")
H2_g = Species.create_gas("H2_g")
O2_g = Species.create_gas("O2_g")
CO_g = Species.create_gas("CO_g")
CO2_g = Species.create_gas("CO2_g")
CH4_g = Species.create_gas("CH4_g")
N2_g = Species.create_gas("N2_g")
NH3_g = Species.create_gas("NH3_g")
S2_g = Species.create_gas("S2_g")
H2S_g = Species.create_gas("H2S_g")
SO2_g = Species.create_gas("SO2_g")
SO_g = Species.create_gas("SO_g")
# Cl is very low abundance in the atmosphere so exclude
# Cl2_g = Species.create_gas("Cl2_g")
H2O_l = Species.create_condensed("H2O_l")
C_cr = Species.create_condensed("C_cr")
S_alpha = Species.create_condensed("S_alpha")
# No thermodata below 368.3 K
#S_beta = Species.create_condensed("S_beta")
# No thermodata below 388.36 K
#S_l = Species.create_condensed("S_l")

species = (H2_g, H2O_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g, S2_g, H2S_g, SO2_g, SO_g, # Cl2_g,
          H2O_l, C_cr, S_alpha) #, S_beta, S_l)

In [33]:
solver_parameters = SolverParameters.create(species, optx.Newton, atol=1.0e-8, rtol=1.0e-8)

Get the atmospheric element abundances from the model with solubilities

In [34]:
interior_atmosphere_cool = InteriorAtmosphere(species)

trappist1e = Planet(surface_temperature=cool_surface_temperature, planet_mass=planet_mass,
    surface_radius=5.861E6, mantle_melt_fraction=0.0)

withsol_dict = output_withsol.asdict()

# Create a dictionary with the series. Only the CHON constraints are uncommented
mass_constraints = {
    "H": withsol_dict["element_H"]["atmosphere_mass"],
    "S": withsol_dict["element_S"]["atmosphere_mass"],
    "N": withsol_dict["element_N"]["atmosphere_mass"],
    #"O": withsol_dict["element_O"]["atmosphere_mass"],
    "C": withsol_dict["element_C"]["atmosphere_mass"],
    #"Cl": withsol_dict["element_Cl"]["atmosphere_mass"]
}

# Initial solution guess number density (molecules/m^3)
# In practice, I run a smaller model to get a sense of the average number densities, and then tweak
# these estimates accordingly.
initial_log_number_density = 50 * np.ones(len(species), dtype=np.float_)
#initial_log_number_density[7] = 20
#initial_log_number_density[9] = 20
#initial_log_number_density[10] = 20
# Initial solution guess species stability
# A large negative initial guess appears to help the solver.
initial_log_stability = -140 * np.ones_like(initial_log_number_density)

# Precompile
interior_atmosphere_cool.initialise_solve(
    planet=trappist1e,
    initial_log_number_density=initial_log_number_density,
    initial_log_stability=initial_log_stability,
    mass_constraints=mass_constraints,
    solver_parameters=solver_parameters
)

output_cool = interior_atmosphere_cool.solve()

# Write the complete solution to Excel
output_cool.to_excel("t1e_coolT")

[16:16:48 - atmodeller.classes             - INFO     ] - species = ['H2_g', 'H2O_g', 'O2_g', 'CO_g', 'CO2_g', 'CH4_g', 'N2_g', 'H3N_g', 'S2_g', 'H2S_g', 'O2S_g', 'OS_g', 'H2O_l', 'C_cr', 'S_alpha']
[16:16:48 - atmodeller.classes             - INFO     ] - reactions = {0: '3.0 H2_g + 1.0 CO_g = 1.0 H2O_g + 1.0 CH4_g',
 1: '2.0 H2O_g = 2.0 H2_g + 1.0 O2_g',
 2: '1.5 H2_g + 0.5 N2_g = 1.0 H3N_g',
 3: '1.0 H2O_g + 1.0 CO_g = 1.0 H2_g + 1.0 CO2_g',
 4: '1.0 H2_g + 0.5 S2_g = 1.0 H2S_g',
 5: '2.0 H2O_g + 0.5 S2_g = 2.0 H2_g + 1.0 O2S_g',
 6: '1.0 H2O_g + 0.5 S2_g = 1.0 H2_g + 1.0 OS_g',
 7: '1.0 H2O_g = 1.0 H2O_l',
 8: '1.0 H2_g + 1.0 CO_g = 1.0 H2O_g + 1.0 C_cr',
 9: '0.5 S2_g = 1.0 S_alpha'}
[16:16:48 - atmodeller.output              - INFO     ] - Found O2_g so back-computing log10 shift for fO2
[16:16:55 - atmodeller.classes             - INFO     ] - Compile time: 7.152158 seconds
[16:17:00 - atmodeller.classes             - INFO     ] - Execution time: 4.627630 seconds
[16:17:00 - atm